<a href="https://colab.research.google.com/github/ShreyJ1729/EVA6-TSAI/blob/main/05-CodingDrillDown/Iteration_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # 28x28x1 --> 28x28x10 --> 26x26x16 --> 24x24x16 --> 12x12x16
        self.convblock1 = nn.Sequential(
            nn.Conv2d(1, 10, 3, padding=1),
            nn.ReLU(),

            nn.Conv2d(10, 16, 3),
            nn.ReLU(),

            nn.Conv2d(16, 16, 3),
            nn.ReLU(),

            nn.MaxPool2d(2, 2)
        )

        # 12x12x16 --> 12x12x32 --> 12x12x32 --> 6x6x16
        self.convblock2 = nn.Sequential(
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),

            nn.Conv2d(32, 32, 3, padding=1),
            nn.ReLU(),

            nn.Conv2d(32, 16, 3, padding=1),
            nn.ReLU(),

            nn.MaxPool2d(2, 2)
        )
        
        # 6x6x16 --> 6x6x10 --> 1x1x10
        self.convblock3 = nn.Sequential(
            nn.Conv2d(16, 10, 3, padding=1),
            nn.AvgPool2d(6)
        )

        self.fcblock = nn.Linear(10, 10)

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = x.view(-1, 10)
        x = self.fcblock(x)
        return F.log_softmax(x, dim=1)

In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]             100
              ReLU-2           [-1, 10, 28, 28]               0
            Conv2d-3           [-1, 16, 26, 26]           1,456
              ReLU-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           2,320
              ReLU-6           [-1, 16, 24, 24]               0
         MaxPool2d-7           [-1, 16, 12, 12]               0
            Conv2d-8           [-1, 32, 12, 12]           4,640
              ReLU-9           [-1, 32, 12, 12]               0
           Conv2d-10           [-1, 32, 12, 12]           9,248
             ReLU-11           [-1, 32, 12, 12]               0
           Conv2d-12           [-1, 16, 12, 12]           4,624
             ReLU-13           [-1, 16, 12, 12]               0
        MaxPool2d-14             [-1, 1

In [ ]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [ ]:
from tqdm.notebook import tqdm
def train(model, device, train_loader, optimizer, epoch):
    epoch_losses = []
    model.train()
    pbar = tqdm(train_loader, leave=False)
    correct = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

        epoch_losses.append(loss.item())
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

        tqdm.write(f'\r[Epoch {epoch}] loss={round(sum(epoch_losses)/len(epoch_losses), 4)}', end="")
    tqdm.write('\r[Epoch {}]\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(epoch, 
        round(sum(epoch_losses)/len(epoch_losses), 4), correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    tqdm.write('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(15):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

[Epoch 0]
Train set: Average loss: 2.3028, Accuracy: 6520/60000 (10.87%)
Test set: Average loss: 2.3011, Accuracy: 1135/10000 (11.35%)



[Epoch 1]
Train set: Average loss: 2.2993, Accuracy: 6862/60000 (11.44%)
Test set: Average loss: 2.2795, Accuracy: 1464/10000 (14.64%)



[Epoch 2]
Train set: Average loss: 1.3965, Accuracy: 29647/60000 (49.41%)
Test set: Average loss: 0.3812, Accuracy: 8805/10000 (88.05%)



[Epoch 3]
Train set: Average loss: 0.2409, Accuracy: 55522/60000 (92.54%)
Test set: Average loss: 0.1474, Accuracy: 9553/10000 (95.53%)



[Epoch 4]
Train set: Average loss: 0.1451, Accuracy: 57307/60000 (95.51%)
Test set: Average loss: 0.1439, Accuracy: 9571/10000 (95.71%)



...

[Epoch 10]
Train set: Average loss: 0.0538, Accuracy: 59011/60000 (98.35%)
Test set: Average loss: 0.0583, Accuracy: 9813/10000 (98.13%)



[Epoch 11]
Train set: Average loss: 0.0490, Accuracy: 59090/60000 (98.48%)
Test set: Average loss: 0.0464, Accuracy: 9866/10000 (98.66%)



[Epoch 12]
Train set: Average loss: 0.0440, Accuracy: 59190/60000 (98.65%)
Test set: Average loss: 0.0397, Accuracy: 9875/10000 (98.75%)



[Epoch 13]
Train set: Average loss: 0.0419, Accuracy: 59205/60000 (98.67%)
Test set: Average loss: 0.0455, Accuracy: 9859/10000 (98.59%)



[Epoch 14]
Train set: Average loss: 0.0404, Accuracy: 59237/60000 (98.73%)
Test set: Average loss: 0.0471, Accuracy: 9846/10000 (98.46%)



Target:
- Get basic working code

Results:
- 98.75% accuracyß
- ~23k parameters
- Way too many parameters
- Model converged quite slowly (first epoch only 11% training accuracy)
- Minor overfitting near last epochs

Analysis:
- Add BatchNorm for faster convergence
- Reduce model number of parameters